    000_16_230114_141_subway_transer_01
    
    data : 004_수도권4호선_주소데이터_20221122.csv
     save : subway_info_04.csv
     display : subway_info_04.html

In [16]:
#1. package load
import requests
import json
import os
import math
import time
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap
print('슝~')

슝~


In [17]:
#2. data load  ...SGIS data 가져오기 # SGIS API 사이트의 본인의 consumer_key, consumer_secret를 입력합니다.
consumer_key = '2f74504a63224ee09f2b'       # 서비스ID
consumer_secret =  '63a0ffef893e43a48ae2' # 보안Key


In [18]:
#2. data load  ... SGIS data 가져오기 # SGIS ACCESS TOKEN을 리턴 해주는 함수
def get_access_token(consumer_key, consumer_secret) -> None:
    temp_res = requests.get(
        url="https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json",
        params={
            "consumer_key": consumer_key,
            "consumer_secret": consumer_secret
        }
    )

    access_token = temp_res.json()["result"]["accessToken"]
   
    return access_token
print('슝~')

슝~


In [19]:
#path = 'c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/001_수도권1호선_주소데이터_20221122.csv'

subway_info_df = pd.read_csv('c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_addr/004_수도권4호선_주소데이터_20221122.csv',encoding='cp949')
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소
0,코레일,4호선,경마공원,경기도 과천시 과천동 654,경기도 과천시 경마공원대로 105
1,코레일,4호선,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784
2,코레일,4호선,과천,경기도 과천시 별양동 2,경기도 과천시 별양로 177
3,코레일,4호선,금정,경기도 군포시 금정동 613 금정역,경기도 군포시 군포로 750(금정동)
4,코레일,4호선,대공원,경기도 과천시 과천동 727-1,경기도 과천시 대공원대로 50


In [20]:
subway_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  48 non-null     object
 1   선명       48 non-null     object
 2   역명       48 non-null     object
 3   지번주소     48 non-null     object
 4   도로명주소    48 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


In [21]:
# subway_info_df['주소'] = subway_info_df['주소'].astype(str)

In [22]:
#5  university   # 주소를 입력해 주소의 위도, 경도를 가지고 오는 함수
def get_adress_x_y(address) -> None:
    # SGIS access_token 가져오기
    access_token = get_access_token(consumer_key , consumer_secret)

    url="https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params={
        "accessToken": access_token,
        "address": address
    }

    temp_res = requests.get(
        url=url,
        params=params
    ).json()["result"]["resultdata"]

    return temp_res[0]["x"], temp_res[0]["y"]

print('슝~')

슝~


In [23]:
#5  university .. 주소 기반 위도,경도 추가
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_adress_x_y(row["도로명주소"])
     
        subway_info_df.loc[_, "x"] = x
        subway_info_df.loc[_, "y"] = y
    except:
        print(row["도로명주소"] + '은(는) 검색에 실패했습니다')

subway_info_df.head()


,철도운영기관명,선명,역명,지번주소,도로명주소,x,y
0,코레일,4호선,경마공원,경기도 과천시 과천동 654,경기도 과천시 경마공원대로 105,956484.100275131,1938442.72191855
1,코레일,4호선,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784,940026.184624752,1924417.61808282
2,코레일,4호선,과천,경기도 과천시 별양동 2,경기도 과천시 별양로 177,955531.10944698,1937256.48118374
3,코레일,4호선,금정,경기도 군포시 금정동 613 금정역,경기도 군포시 군포로 750(금정동),950716.653443351,1930504.44293082
4,코레일,4호선,대공원,경기도 과천시 과천동 727-1,경기도 과천시 대공원대로 50,956259.229185334,1937511.34182009


In [24]:
#5  university  ..# SGIS 좌표변환 API를 활용한 좌표변환 함수
def get_transcoord(src, dst, posX, posY) -> None:
    access_token = get_access_token(consumer_key, consumer_secret)
    url="https://sgisapi.kostat.go.kr/OpenAPI3/transformation/transcoord.json"
    params={
        "accessToken": access_token,
        "src": src, # 현재 좌표체계
        "dst": dst, # 변환할 좌표체계
        "posX": posX, # 변환할 좌표 X값
        "posY":	posY # 변환할 좌표 Y값
    }

    temp_dict = requests.get(
        url=url,
        params=params
    ).json()["result"]

    return temp_dict["posX"], temp_dict["posY"] # 변환된 좌표값 딕셔너리의 value 리턴

print('슝~')


슝~


In [25]:
#5  university  .. 좌표계가 UTM-K이므로 WSG84좌표계로 변환
for _, row in subway_info_df.iterrows():
    try:
        x, y = get_transcoord("EPSG:5179", "EPSG:4326", row["x"], row["y"])
    
        subway_info_df.loc[_, "lng"] = x
        subway_info_df.loc[_, "lat"] = y

    except:
        pass
subway_info_df.head()

,철도운영기관명,선명,역명,지번주소,도로명주소,x,y,lng,lat
0,코레일,4호선,경마공원,경기도 과천시 과천동 654,경기도 과천시 경마공원대로 105,956484.100275131,1938442.72191855,127.008051,37.444140
1,코레일,4호선,고잔,경기도 안산시 단원구 고잔동 453-67,경기도 안산시 단원구 중앙대로 784,940026.184624752,1924417.61808282,126.823141,37.316808
2,코레일,4호선,과천,경기도 과천시 별양동 2,경기도 과천시 별양로 177,955531.10944698,1937256.48118374,126.997350,37.433403
3,코레일,4호선,금정,경기도 군포시 금정동 613 금정역,경기도 군포시 군포로 750(금정동),950716.653443351,1930504.44293082,126.943382,37.372300
4,코레일,4호선,대공원,경기도 과천시 과천동 727-1,경기도 과천시 대공원대로 50,956259.229185334,1937511.34182009,127.005564,37.435735


In [26]:
#5  university  .. 시각화 .. 초기 중심값 좌표가 들어가야 합니다.

# 서울시 중심부 위도, 경도
center = [37.541, 126.986]

# map이 서울 중심부에 위치하고, zoom 레벨은 12로 시작합니다.
map = folium.Map(
    location=center,
    zoom_start=12

)

print('슝~')

슝~


In [27]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in subway_info_df.iterrows():
    try:
        folium.Marker(
            location=[subway_info_df.loc[idx, "lat"], subway_info_df.loc[idx, "lng"]],
            popup=subway_info_df.loc[idx, "역명"],
            icon=folium.Icon(color="cadetblue", icon="flag")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

슝~


In [28]:
#5 seoul university 
map

In [29]:
map.save("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/003_map_xml/subway_info_04.html")


In [30]:
#apt_df.to_excel("C:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/data/apt_cn_sung.xls", index = False)
# subway_info_df.to_csv("c:/Users/User/000_my_jupyter/900_aiffel/0000_AIFFELTHON/3김연수/002_data_dir/003_search_data/subway_info_02.csv", index = False)
subway_info_df.to_csv("subway_info_04.csv", index = False)
